# Use Python API to automate AutoAI deployment lifecycle

This notebook contains the steps and code to demonstrate support of AI Lifecycle features of the AutoAI model in watsonx.ai service. It contains steps and code to work with [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) SDK available in PyPI repository. It also introduces commands for training, persisting and deploying model, scoring it, updating the model and redeploying it.

Some familiarity with Python is helpful. This notebook uses Python 3.12.


## Learning goals

The learning goals of this notebook are:

-  List all deprecated and unsupported deployments.
-  Identify AutoAI models that need to be retrained.
-  Work with watsonx.ai experiments to re-train AutoAI models.
-  Persist an updated AutoAI model in watsonx.ai repository.
-  Redeploy model in-place.
-  Score sample records using client library.


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Deployments state check](#state)
3. [Identification of model requiring retraining](#identify)
4. [Experiment re-run](#run)
5. [Persist trained AutoAI model](#persistence)
6. [Redeploy and score new version of the model](#redeploy)
7. [Clean up](#cleanup)
8. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, contact with your Cloud Pak for Data administrator and ask for your account credentials.

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U wget | tail -n 1
%pip install "scikit-learn==1.6.1" | tail -n 1
%pip install -U autoai-libs | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [2]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

Enter your watsonx.ai password and hit enter:  ········


#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).


You can use the `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

Extract all space IDs

In [5]:
space_ids = [
    space["metadata"]["id"] for space in client.spaces.get_details()["resources"]
]

space_ids[:5]

['dca8e5ae-bf7f-4903-b200-65bfa21fd4c5',
 'ee2821f4-cb2a-412a-b093-0106204a3e8e',
 'f6d27946-2381-4abe-a620-7e4b1afe8a16',
 '3813019e-e0d3-4b96-92ac-04a30af93744',
 '5a9af126-1850-4b69-a1ed-cf1d66e73176']

<a id="state"></a>
## 2. Deployments state check
Iterate over spaces and search for `deprecated` and `unsupported` deployments. Next, identify models requiring re-training.

In [6]:
from ibm_watsonx_ai.lifecycle import SpecStates


for space_id in space_ids[:5]:
    client.set.default_space(space_id)
    print(f"****** SPACE {space_id} ******")
    print(client.deployments.get_details(spec_state=SpecStates.DEPRECATED))
    print(client.deployments.get_details(spec_state=SpecStates.UNSUPPORTED))

****** SPACE dca8e5ae-bf7f-4903-b200-65bfa21fd4c5 ******
{'resources': []}
{'resources': []}
****** SPACE ee2821f4-cb2a-412a-b093-0106204a3e8e ******
{'resources': []}
{'resources': []}
****** SPACE f6d27946-2381-4abe-a620-7e4b1afe8a16 ******
{'resources': []}
{'resources': []}
****** SPACE 3813019e-e0d3-4b96-92ac-04a30af93744 ******
{'resources': []}
{'resources': []}
****** SPACE 5a9af126-1850-4b69-a1ed-cf1d66e73176 ******
{'resources': []}
{'resources': []}


You can also list deployments under particular space. The output contains `SPEC_STATE` and `SPEC_REPLACEMENT`. Set the working space.

In [7]:
deployment_space_id = "PASTE YOUR SPACE ID HERE"
client.set.default_space(deployment_space_id)

'SUCCESS'

List deployments under this space.

In [8]:
client.deployments.list()

,ID,NAME,STATE,CREATED,ARTIFACT_TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,9c6943cd-e804-43b6-9d49-3c094e92146b,Credit Risk Deployment AutoAI,ready,2025-05-23T11:04:25.856Z,model,supported,
1,65d48c40-a230-4839-80a1-8836dd935566,AutoAI credit-risk deployment,ready,2025-05-23T10:52:37.530Z,model,supported,
2,977b4d07-650a-4243-a781-d144341b417f,AutoAI credit-risk deployment,ready,2025-05-23T10:45:49.010Z,model,supported,


<a id="identify"></a>
## 3. Identification of model requiring retraining
Pick up deployment of the AutoAI model you wish to retrain. 

**Hint**: You can also do that programatically in the loop sequence over spaces check (`Check the state of your deployments` cell).
**Hint**: You can also use software_specification information (model details) to identify models and deployments that are not yet deprecated but can be retrained (updated software specification is available).

In [9]:
deployment_id = "PASTE YOUR DEPLOYMENT ID HERE"

deployment_details = client.deployments.get_details(deployment_id)
deployed_model_id = deployment_details["entity"]["asset"]["id"]

deployed_model_id

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.


'269f7f9a-920c-4ef4-b021-ef5786ae709a'

#### Extract the deployed model's details (including the pipeline information).

In [10]:
import json

deployed_model_details = client.repository.get_model_details(deployed_model_id)
deployed_pipeline_id = deployed_model_details["entity"]["pipeline"]["id"]

deployed_pipeline_details = client.repository.get_details(deployed_pipeline_id)
experiment_params = deployed_pipeline_details["entity"]["document"]["pipelines"][0]["nodes"][0]["parameters"]

optimization_params = experiment_params["optimization"]

print("Experiment parameters:", json.dumps(experiment_params, indent=2))
print("Optimization parameters:", json.dumps(optimization_params, indent=2))

Experiment parameters: {
  "drop_duplicates": true,
  "encoding": "utf-8",
  "input_file_separator": ",",
  "optimization": {
    "compute_pipeline_notebooks_flag": true,
    "daub_adaptive_subsampling_max_mem_usage": 9000000000,
    "label": "Risk",
    "learning_type": "binary",
    "retrain_on_holdout": true,
    "run_cognito_flag": true,
    "scorer_for_ranking": "roc_auc"
  },
  "output_logs": true,
  "stage_flag": true
}
Optimization parameters: {
  "compute_pipeline_notebooks_flag": true,
  "daub_adaptive_subsampling_max_mem_usage": 9000000000,
  "label": "Risk",
  "learning_type": "binary",
  "retrain_on_holdout": true,
  "run_cognito_flag": true,
  "scorer_for_ranking": "roc_auc"
}


#### Find the AutoAI experiment runs matching the extracted pipeline

Extract the project ID where the training took place.

**Tip:** For more information about using AutoAI with projects, see [this sample notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/experiments/autoai/Use%20AutoAI%20with%20Watson%20Studio%20project.ipynb).

**Note:** If the training took place in a space, please update accordingly.

In [11]:
try:
    training_project_id = deployed_pipeline_details["metadata"]["tags"][0].split(".")[1]
except LookupError:
    training_project_id = input("Please enter your project_id (hit enter): ")

Please enter your project_id (hit enter):  9cf5ee36-da98-4856-be9f-a04df0d43f7b


#### Extract AutoAI experiment `training_id`

The `training_id` is available in model's details.

In [12]:
run_id = deployed_model_details["entity"]["training_id"]
print("AutoAI experiment training_id found in model details:", run_id)

AutoAI experiment training_id found in model details: 69e7991d-eacb-4f51-bbd1-631fa91d17bd


<a id="run"></a>
## 4. Experiment re-run 
Set the training `project_id` (where data asset resides) to retrain AutoAI models.


In [13]:
from ibm_watsonx_ai.experiment import AutoAI

experiment = AutoAI(credentials, project_id=training_project_id)
optimizer = experiment.runs.get_optimizer(run_id=run_id)

In [14]:
from ibm_watsonx_ai.utils.autoai.errors import TestDataNotPresent

training_data_reference = optimizer.get_data_connections()
try:
    test_data_reference = optimizer.get_test_data_connections()
except TestDataNotPresent:
    test_data_reference = None

User defined (test / holdout) data is not present for this AutoAI experiment.
Reason: User specified test data was not present in this experiment. Try to use 'with_holdout_split' parameter for original training_data_references to retrieve test data.


In [15]:
train_details = optimizer.fit(
    training_data_references=training_data_reference,
    test_data_references=test_data_reference,
)

Training job edc7616d-a2c9-465a-9dab-d01574fba1a5 completed: 100%|████████| [02:18<00:00,  1.39s/it]


### Explore experiment's results
Connect to finished experiment and preview the results.

In [16]:
optimizer.summary()

,Enhancements,Estimator,training_roc_auc_(optimized),holdout_average_precision,holdout_log_loss,training_accuracy,holdout_roc_auc,training_balanced_accuracy,training_f1,holdout_precision,training_average_precision,training_log_loss,holdout_recall,training_precision,holdout_accuracy,holdout_balanced_accuracy,training_recall,holdout_f1
Pipeline Name,,,,,,,,,,,,,,,,,,
Pipeline_2,HPO,XGBClassifier,0.853281,0.479185,0.425331,0.800358,0.832534,0.748267,0.857836,0.816216,0.916755,0.428845,0.909639,0.814358,0.803607,0.751226,0.906382,0.860399
Pipeline_1,,XGBClassifier,0.848451,0.466398,0.356549,0.796120,0.829125,0.749861,0.852969,0.848066,0.912986,0.439419,0.924699,0.818963,0.839679,0.797679,0.890274,0.884726
Pipeline_6,,SnapBoostingMachineClassifier,0.850092,0.468067,0.403035,0.755522,0.825103,0.745869,0.808007,0.893333,0.915222,0.457473,0.807229,0.844414,0.807615,0.807806,0.775171,0.848101
Pipeline_3,"HPO, FE",XGBClassifier,0.852998,0.480987,0.433601,0.799690,0.823101,0.744796,0.858098,0.807487,0.915761,0.428990,0.909639,0.810770,0.795591,0.739250,0.911415,0.855524
Pipeline_7,HPO,SnapBoostingMachineClassifier,0.851431,0.477125,0.456636,0.749054,0.822352,0.747928,0.798939,0.883803,0.916453,0.466879,0.756024,0.853730,0.771543,0.779210,0.751345,0.814935
Pipeline_4,"HPO, FE, HPO",XGBClassifier,0.853515,0.483860,0.444985,0.800582,0.820594,0.743488,0.859371,0.796296,0.916422,0.429937,0.906627,0.808763,0.783567,0.722774,0.916783,0.847887
Pipeline_5,"HPO, FE, HPO, Ensemble",BatchedTreeEnsembleClassifier(XGBClassifier),0.853515,0.483860,0.444985,0.800582,0.820594,0.743488,0.859371,0.796296,0.916422,0.429937,0.906627,0.808763,0.783567,0.722774,0.916783,0.847887
Pipeline_9,"HPO, FE, HPO",SnapBoostingMachineClassifier,0.854561,0.471638,0.424094,0.762884,0.819223,0.753056,0.814332,0.882943,0.917152,0.455770,0.795181,0.848812,0.793587,0.792800,0.782893,0.836767
Pipeline_10,"HPO, FE, HPO, Ensemble",BatchedTreeEnsembleClassifier(SnapBoostingMach...,0.854561,0.471638,0.424094,0.762884,0.819223,0.753056,0.814332,0.882943,0.917152,0.455770,0.795181,0.848812,0.793587,0.792800,0.782893,0.836767


### Evaluate the best model locally

Load the model for test purposes.

**Hint:** The best model is returned automatically if no `pipeline_name` provided.

In [17]:
pipeline_name = "Pipeline_4"
pipeline_model = optimizer.get_pipeline(pipeline_name=pipeline_name, astype="sklearn")
pipeline_model

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('float32_transform_139766486739008',
                                                 Pipeline(steps=[('numpycolumnselector',
                                                                  NumpyColumnSelector(columns=[0,
                                                                                               1,
                                                                                               2,
                                                                                               3,
                                                                                               5,
                                                                                               6,
                                                                                               7,
                                                                                               8,
                                                                                               9,
                                                                                               10,
                                                                                               11,
                                                                                               12,
                                                                                               13,
                                                                                               14,
                                                                                               15,
                                                                                               16,
                                                                                               17,
                                                                                               18,
                                                                                               19])),
                                                                 ('compressstrings',
                                                                  CompressStrings(compress_type='hash',
                                                                                  dtypes_list=['char_str',
                                                                                               'float_int_num',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'c...
                               feature_types=None, gamma=1, gpu_id=None,
                               grow_policy=None, importance_type='gain',
                               interaction_constraints=None,
                               learning_rate=0.0224493228186446, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=4,
                               max_leaves=None, min_child_weight=19,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=204, n_jobs=4,
                               nthread=None, ...))])

This cell constructs the cell scorer based on the experiment metadata.

In [18]:
from sklearn.metrics import get_scorer

scorer = get_scorer(optimization_params["scorer_for_ranking"])

#### Read the train and holdout data.

**Hint:** You can also use external test dataset.

In [19]:
connection = optimizer.get_data_connections()[0]
train_X, test_X, train_y, test_y = connection.read(with_holdout_split=True)

#### Calculate the score

In [20]:
score = scorer(pipeline_model, test_X.values, test_y.values)
print(score)

0.845104970781329


<a id="persistence"></a>

## 5. Store the model in repository

Provide `pipeline_name` and `training_id`.

In [21]:
client.set.default_project(training_project_id)

Unsetting the space_id ...


'SUCCESS'

In [22]:
model_metadata = {
    client.repository.ModelMetaNames.NAME: "{0} - {1} - {2}".format(
        deployed_pipeline_details["metadata"]["name"],
        pipeline_name,
        pipeline_model.get_params()["steps"][-1][0],
    )
}
published_model = client.repository.store_model(
    model=pipeline_name,
    meta_props=model_metadata,
    training_id=train_details["metadata"]["id"],
)
updated_model_id = client.repository.get_model_id(published_model)
print("Re-trained model id", updated_model_id)

Re-trained model id 8683b3a7-bb11-46da-b46f-52355acb8c18


List stored models.

In [23]:
client.repository.list_models()

,ID,NAME,CREATED,TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,8683b3a7-bb11-46da-b46f-52355acb8c18,Credit Risk Prediction - AutoAI - Pipeline_4 -...,2025-05-23T11:23:57Z,wml-hybrid_0.1,supported,
1,ff4f3ce2-a0bc-4d5b-b6d5-4bee24bac318,P1,2025-05-23T11:04:22Z,wml-hybrid_0.1,supported,


<a id="redeploy"></a>
## 6. Redeploy and score new version of the model

In this section, you'll learn how to redeploy new version of the model by using the watsonx.ai Client.

**Hint:** As a best practice please consider using the test space before moving to production.

```
promote(asset_id: str, source_project_id: str, target_space_id: str, rev_id: str = None)
```

### Promote model to deployment space

In [24]:
promoted_model_id = client.spaces.promote(
    asset_id=updated_model_id,
    source_project_id=training_project_id,
    target_space_id=deployment_space_id,
)

Check current deployment details before update.

In [ ]:
client.set.default_space(deployment_space_id)
print(json.dumps(client.deployments.get_details(deployment_id), indent=2))

### Update the deployment with new model
**Note:** The update is asynchronous.

In [26]:
metadata = {
    client.deployments.ConfigurationMetaNames.ASSET: {
        "id": promoted_model_id,
    }
}

updated_deployment = client.deployments.update(deployment_id, changes=metadata)

Since ASSET is patched, deployment need to be restarted. 


########################################################################

Deployment update for id: '9c6943cd-e804-43b6-9d49-3c094e92146b' started

########################################################################


updating.......
ready


---------------------------------------------------------------------------------------------
Successfully finished deployment update, deployment_id='9c6943cd-e804-43b6-9d49-3c094e92146b'
---------------------------------------------------------------------------------------------




Wait for the deployment update: 

In [27]:
import time

status = None
while status not in ("ready", "failed"):
    time.sleep(2)
    deployment_details = client.deployments.get_details(deployment_id)
    status = deployment_details["entity"]["status"].get("state")
    print(".", status, end=" ")

print("\nDeployment update finished with status: ", status)

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
. ready 
Deployment update finished with status:  ready


#### Get updated deployment details

In [ ]:
print(json.dumps(client.deployments.get_details(deployment_id), indent=2))

### Score updated model
Create sample payload and score the deployed model.

In [29]:
scoring_payload = {"input_data": [{"values": test_X[:3]}]}

Use client.deployments.score() method to run scoring.

In [30]:
predictions = client.deployments.score(deployment_id, scoring_payload)

In [31]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction",
        "probability"
      ],
      "values": [
        [
          "No Risk",
          [
            0.5360556244850159,
            0.46394437551498413
          ]
        ],
        [
          "No Risk",
          [
            0.9565750956535339,
            0.04342489689588547
          ]
        ],
        [
          "No Risk",
          [
            0.57205730676651,
            0.42794269323349
          ]
        ]
      ]
    }
  ]
}


<a id="cleanup"></a>
## 7. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- models
- deployments

please follow up this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 8. Summary and next steps

You successfully completed this notebook! You learned how to use scikit-learn machine learning as well as watsonx.ai for model creation and deployment.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski, PhD**, is a Senior Technical Staff Member at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

**Dorota Lączak**, Python Software Developer in Watson Machine Learning AutoAI at IBM

Copyright © 2023-2025 IBM. This notebook and its source code are released under the terms of the MIT License.